# BentoML Example: SKLearn to ONNX model


BentoML is an open-source framework for machine learning **model serving**, aiming to **bridge the gap between Data Science and DevOps.**

Data Scientists can easily package their models trained with any ML framework using BentoMl and reproduce the model for serving in production. BentoML helps with managing packaged models in the BentoML format, and allows DevOps to deploy them as online API serving endpoints or offline batch inference jobs, on any cloud platform.

Before reading this example project, be sure to check out the [Getting started guide](https://github.com/bentoml/BentoML/blob/master/guides/quick-start/bentoml-quick-start-guide.ipynb) to learn about the basic concepts in BentoML.

This example notebooks demonstrates how to build SK model and packed as ONNX model for BentoML

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=onnx&ea=onnx-sk-iris-classifier&dt=onnx-sk-iris-classifier)

In [1]:
!pip install -q bentoml "skl2onnx==1.7.0" "onnx==1.7.0" "onnxmltools==1.7.0"

  Using cached skl2onnx-1.7.0-py2.py3-none-any.whl (191 kB)
  Using cached onnxmltools-1.7.0-py2.py3-none-any.whl (252 kB)
  Using cached onnxconverter_common-1.7.0-py2.py3-none-any.whl (64 kB)
  Using cached keras2onnx-1.7.0-py3-none-any.whl (96 kB)
  Using cached fire-0.3.1.tar.gz (81 kB)
  Created wheel for fire: filename=fire-0.3.1-py2.py3-none-any.whl size=111005 sha256=771d43d8451dfcbda6c1066b6e619b3f1ddec020d8ecc3fb0d8c0e70bbf8e526
  Stored in directory: /home/bentoml/.cache/pip/wheels/95/38/e1/8b62337a8ecf5728bdc1017e828f253f7a9cf25db999861bec
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4830 sha256=4551679660493ab6f79c2ab0add35ed6f744f524e43039ab340d2abc7d796707
  Stored in directory: /home/bentoml/.cache/pip/wheels/3f/e3/ec/8a8336ff196023622fbcb36de0c5a5c218cbb24111d1d4c7f2
Successfully built fire termcolor


In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)

from sklearn.linear_model import LogisticRegression
clr = LogisticRegression()
clr.fit(X_train, y_train)

LogisticRegression()

In [2]:
clr.predict([[5.1, 3.5, 1.4, 0.2]])

array([0])

In [3]:
%%writefile iris_classifier_onnx.py

import numpy
import bentoml
from bentoml.adapters import DataframeInput
from bentoml.frameworks.onnx import OnnxModelArtifact

@bentoml.artifacts([OnnxModelArtifact('model')])
@bentoml.env(infer_pip_packages=True)
class IrisClassifierOnnx(bentoml.BentoService):
    
    @bentoml.api(input=DataframeInput())
    def predict(self, df):
        input_data = df.to_numpy().astype(numpy.float32)
        input_name = self.artifacts.model.get_inputs()[0].name
        output_name = self.artifacts.model.get_outputs()[0].name
        outputs = numpy.zeros(input_data.shape[0])
        for i in range(input_data.shape[0]):
            outputs[i] = self.artifacts.model.run([output_name], {input_name: input_data[i: i + 1]})[0]
        return outputs

Writing iris_classifier_onnx.py


In [4]:
from iris_classifier_onnx import IrisClassifierOnnx
from skl2onnx.common.data_types import FloatTensorType
import onnxmltools



initial_type = [('float_input', FloatTensorType([None, 4]))]
onnx_model = onnxmltools.convert_sklearn(clr, initial_types=initial_type)

svc = IrisClassifierOnnx()
svc.pack('model', onnx_model)

saved_path = svc.save()

[2020-09-08 21:01:18,365] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-08 21:01:21,001] INFO - Detected non-PyPI-released BentoML installed, copying local BentoML modulefiles to target saved bundle path..


no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'benchmark'


UPDATING BentoML-0.8.6+36.gab94ebfb/bentoml/_version.py
set BentoML-0.8.6+36.gab94ebfb/bentoml/_version.py to '0.8.6+36.gab94ebfb'
[2020-09-08 21:01:25,458] INFO - BentoService bundle 'IrisClassifierOnnx:20200908210120_30DE0E' saved to: /Users/bozhaoyu/bentoml/repository/IrisClassifierOnnx/20200908210120_30DE0E


## REST API Model Serving


To start a REST API model server with the BentoService saved above, use the bentoml serve command:

In [2]:
!bentoml serve IrisClassifierOnnx:latest --enable-microbatch

[2020-08-04 09:58:22,230] INFO - Getting latest version IrisClassifierOnnx:20200804094903_8746D7
[2020-08-04 09:58:22,230] INFO - Starting BentoML API server in development mode..
[2020-08-04 09:58:23,377] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-08-04 09:58:23,405] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.1, but loading from BentoML version 0.8.3+47.g5daa71b
[2020-08-04 09:58:23,406] WARNING - Saved BentoService Python version mismatch: loading BentoService bundle created with Python version 3.7.4, but current environment version is 3.6.10.
[2020-08-04 09:58:24,464] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.1, but loading from BentoML version 0.8.

If you are running this notebook from Google Colab, you can start the dev server with `--run-with-ngrok` option, to gain acccess to the API endpoint via a public endpoint managed by [ngrok](https://ngrok.com/):

In [ ]:
!bentoml serve IrisClassifierOnnx:latest --run-with-ngrok

#### Send prediction requeset to the REST API server

```
curl -X POST \
  http://localhost:5000/predict \
  -H 'Content-Type: application/json' \
  -d '[[5.1, 3.5, 1.4, 0.2]]'
```



## Containerize model server with Docker


One common way of distributing this model API server for production deployment, is via Docker containers. And BentoML provides a convenient way to do that.

Note that docker is **not available in Google Colab**. You will need to download and run this notebook locally to try out this containerization with docker feature.

If you already have docker configured, simply run the follow command to product a docker container serving the IrisClassifier prediction service created above:

In [8]:
!bentoml containerize IrisClassifierOnnx:latest

[2020-09-08 21:02:13,762] INFO - Getting latest version IrisClassifierOnnx:20200908210120_30DE0E
Found Bento: /Users/bozhaoyu/bentoml/repository/IrisClassifierOnnx/20200908210120_30DE0E
[2020-09-08 21:02:13,800] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-08 21:02:13,814] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.6, but loading from BentoML version 0.8.6+36.gab94ebfb
Tag not specified, using tag parsed from BentoService: 'irisclassifieronnx:20200908210120_30DE0E'
Building Docker image irisclassifieronnx:20200908210120_30DE0E from IrisClassifierOnnx:latest 
-we in here
processed docker file
(None, None)
root in create archive /Users/bozhaoyu/bentoml/repository/IrisClassifierOnnx/20200908210120_30DE0E ['Dockerfile', 'Iris

|Requirement already satisfied: psutil in /opt/conda/lib/python3.7/site-packages (from bentoml==0.8.6->-r ./requirements.txt (line 3)) (5.7.2)
\Requirement already satisfied: packaging in /opt/conda/lib/python3.7/site-packages (from bentoml==0.8.6->-r ./requirements.txt (line 3)) (20.4)
-Requirement already satisfied: six>=1.9 in /opt/conda/lib/python3.7/site-packages (from protobuf->onnxruntime->-r ./requirements.txt (line 1)) (1.14.0)
/Requirement already satisfied: thriftpy2>=0.4.0 in /opt/conda/lib/python3.7/site-packages (from py-zipkin->bentoml==0.8.6->-r ./requirements.txt (line 3)) (0.4.11)
|Installing collected packages: numpy, onnxruntime, pytz, pandas
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.1
\    Uninstalling numpy-1.19.1:
|      Successfully uninstalled numpy-1.19.1
\Successfully installed numpy-1.18.4 onnxruntime-1.4.0 pandas-1.1.2 pytz-2020.1
\+ for filename in ./bundled_pip_dependencies/*.tar.gz
+ '[' -e ./bundled_pip_dependenc

-Requirement already satisfied, skipping upgrade: psutil in /opt/conda/lib/python3.7/site-packages (from BentoML==0.8.6+36.gab94ebfb) (5.7.2)
/Requirement already satisfied, skipping upgrade: botocore<1.18.0,>=1.17.48 in /opt/conda/lib/python3.7/site-packages (from boto3->BentoML==0.8.6+36.gab94ebfb) (1.17.48)
-  Building wheel for BentoML (PEP 517): finished with status 'done'
/  Created wheel for BentoML: filename=BentoML-0.8.6+36.gab94ebfb-py3-none-any.whl size=4712451 sha256=73fb250e9870b0bc30ede64baa633026a7fd371b7804d89fb5fde4620f36dbdc
  Stored in directory: /tmp/pip-ephem-wheel-cache-dyo1xnkk/wheels/5b/25/f1/96ac959990dff58debbbcd6633150d991d29778500180aea00
/  Building wheel for sqlalchemy-utils (setup.py): finished with status 'done'
  Created wheel for sqlalchemy-utils: filename=SQLAlchemy_Utils-0.36.7-py2.py3-none-any.whl size=93226 sha256=8cb0631ba8096daa9e82e11f8865cd5f2c435e137e6bd8896f4c3a566fe1678c
  Stored in directory: /tmp/pip-ephem-wheel-cache-dyo1xnkk/wheels/

\      Successfully uninstalled BentoML-0.8.6
-Successfully installed BentoML-0.8.6+36.gab94ebfb sqlalchemy-utils-0.36.7
- ---> def3ab24579a
Step 10/15 : ENV PORT 5000
 ---> Running in e5e3c0e8eb8c
/ ---> 72dbee777bb8
Step 11/15 : EXPOSE $PORT
| ---> Running in a54ed8aec1e8
\ ---> 3f199643f78b
Step 12/15 : COPY docker-entrypoint.sh /usr/local/bin/
- ---> ffa40cdc4858
Step 13/15 : RUN chmod +x /usr/local/bin/docker-entrypoint.sh
/ ---> Running in dc1720254890
\ ---> 5245244dfb5c
Step 14/15 : ENTRYPOINT [ "docker-entrypoint.sh" ]
- ---> Running in a596521a5479
/ ---> 8334a0891e76
Step 15/15 : CMD ["bentoml", "serve-gunicorn", "/bento"]
 ---> Running in 1594341560fd
| ---> a72753d5177f
Successfully built a72753d5177f
Successfully tagged irisclassifieronnx:20200908210120_30DE0E
Finished building irisclassifieronnx:20200908210120_30DE0E from IrisClassifierOnnx:latest


In [ ]:
!docker run --rm -p 5000:5000 irisclassifieronnx

## Load saved BentoService

bentoml.load is the API for loading a BentoML packaged model in python:

In [ ]:
from bentoml import load


svc = load(saved_path)

print(svc.predict([[5.1, 3.5, 1.4, 0.2]]))

## Launch inference job from CLI

BentoML cli supports loading and running a packaged model from CLI. With the DataframeInput adapter, the CLI command supports reading input Dataframe data from CLI argument or local csv or json files:

In [7]:
!bentoml run IrisClassifierOnnx:latest predict --input '[[5.1, 3.5, 1.4, 0.2], [5.1, 3.5, 1.4, 0.2]]'

[2020-09-08 21:01:31,260] INFO - Getting latest version IrisClassifierOnnx:20200908210120_30DE0E
[2020-09-08 21:01:31,299] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-08 21:01:31,314] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.6, but loading from BentoML version 0.8.6+36.gab94ebfb
[2020-09-08 21:01:31,790] INFO - Initializing onnxruntime InferenceSession from onnx file:'/Users/bozhaoyu/bentoml/repository/IrisClassifierOnnx/20200908210120_30DE0E/IrisClassifierOnnx/artifacts/model.onnx'
[0.0, 0.0]


# Deployment Options

If you are at a small team with limited engineering or DevOps resources, try out automated deployment with BentoML CLI, currently supporting AWS Lambda, AWS SageMaker, and Azure Functions:
- [AWS Lambda Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_lambda.html)
- [AWS SageMaker Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_sagemaker.html)
- [Azure Functions Deployment Guide](https://docs.bentoml.org/en/latest/deployment/azure_functions.html)

If the cloud platform you are working with is not on the list above, try out these step-by-step guide on manually deploying BentoML packaged model to cloud platforms:
- [AWS ECS Deployment](https://docs.bentoml.org/en/latest/deployment/aws_ecs.html)
- [Google Cloud Run Deployment](https://docs.bentoml.org/en/latest/deployment/google_cloud_run.html)
- [Azure container instance Deployment](https://docs.bentoml.org/en/latest/deployment/azure_container_instance.html)
- [Heroku Deployment](https://docs.bentoml.org/en/latest/deployment/heroku.html)

Lastly, if you have a DevOps or ML Engineering team who's operating a Kubernetes or OpenShift cluster, use the following guides as references for implementating your deployment strategy:
- [Kubernetes Deployment](https://docs.bentoml.org/en/latest/deployment/kubernetes.html)
- [Knative Deployment](https://docs.bentoml.org/en/latest/deployment/knative.html)
- [Kubeflow Deployment](https://docs.bentoml.org/en/latest/deployment/kubeflow.html)
- [KFServing Deployment](https://docs.bentoml.org/en/latest/deployment/kfserving.html)
- [Clipper.ai Deployment Guide](https://docs.bentoml.org/en/latest/deployment/clipper.html)